# Parameter Indexing

For the foreseeable future, parameter indexing is outside of ParamTools' scope. However, indexed parameters are an important part of many modeling projects. Let's take a look at how one might implement indexed parameters with ParamTools.

Setup
--------------------------

The TaxParams parameters will be used to demonstrate parameter indexing. These parameters are based directly on the Tax-Calculator policy parameters. Tax-Calculator does really serious parameter indexing. It blows up values using inflation rates and wage growth rates. This tutorial demonstrates how you can replicate that same level of parameter indexing with ParamTools.

The approach for this tutorial is to build a `IndexedParameters` class on top of `paramtools.Parameters`. This class can then be used by projects that require parameter indexing.

Before we get started, let's make sure that the TaxParams can be loaded.

In [20]:
# first define the compatible data custom field.
from marshmallow import Schema, fields

import paramtools

class CompatibleDataSchema(Schema):
    """
    Schema for Compatible data object
    {
        "compatible_data": {"data1": bool, "data2": bool, ...}
    }
    """

    puf = fields.Boolean()
    cps = fields.Boolean()

class TaxParams(paramtools.Parameters):
    # You need to be in the paramtools/examples/taxparams directory!
    schema = "schema.json"
    defaults = "defaults.json"
    field_map = {"compatible_data": fields.Nested(CompatibleDataSchema())}

params = TaxParams()
print("EITC celing max year: ", max(map(lambda x: x["year"], params._EITC_c)), "\n")
print("EITC ceiling ceiling as dict: ", params._EITC_c)

EITC celing max year:  2018 

EITC ceiling ceiling as dict:  [{'year': 2013, 'value': 487.0, 'EIC': '0kids'}, {'year': 2013, 'value': 3250.0, 'EIC': '1kid'}, {'year': 2013, 'value': 5372.0, 'EIC': '2kids'}, {'year': 2013, 'value': 6044.0, 'EIC': '3+kids'}, {'year': 2014, 'value': 496.0, 'EIC': '0kids'}, {'year': 2014, 'value': 3305.0, 'EIC': '1kid'}, {'year': 2014, 'value': 5460.0, 'EIC': '2kids'}, {'year': 2014, 'value': 6143.0, 'EIC': '3+kids'}, {'year': 2015, 'value': 503.0, 'EIC': '0kids'}, {'year': 2015, 'value': 3359.0, 'EIC': '1kid'}, {'year': 2015, 'value': 5548.0, 'EIC': '2kids'}, {'year': 2015, 'value': 6242.0, 'EIC': '3+kids'}, {'year': 2016, 'value': 506.0, 'EIC': '0kids'}, {'year': 2016, 'value': 3373.0, 'EIC': '1kid'}, {'year': 2016, 'value': 5572.0, 'EIC': '2kids'}, {'year': 2016, 'value': 6269.0, 'EIC': '3+kids'}, {'year': 2017, 'value': 510.0, 'EIC': '0kids'}, {'year': 2017, 'value': 3400.0, 'EIC': '1kid'}, {'year': 2017, 'value': 5616.0, 'EIC': '2kids'}, {'year': 2017

Extend Parameters
---------------------

To get started, the ability to extend parameters through a given year needs to be added to ParamTools. This is done by finding the maximum specifed year for each parameter and duplicating each value object defined at this maximum year for the remaining years. For example, the maximum defined year for `_EITC_c` is 2018. There are four values defined in 2018. These four values will be extended for the remaining years, 2019 to 2028.

```json
[
    {'year': 2018, 'value': 519.0, 'EIC': '0kids'}, 
    {'year': 2018, 'value': 3461.0, 'EIC': '1kid'}, 
    {'year': 2018, 'value': 5716.0, 'EIC': '2kids'}, 
    {'year': 2018, 'value': 6431.0, 'EIC': '3+kids'}
]
```

In [35]:
from collections import defaultdict

import paramtools

class IndexedParameters(paramtools.Parameters):
    
    def extend(self):
        """
        Guarantee that all parameters are defined for each year
        from start year to end year.
        """
        max_allowed_year = max(self._stateless_dim_mesh["year"])
        adjustment = defaultdict(list)
        for param, data in self._data.items():
            # Assume they are sorted by year in ascending order.
            max_year = data["value"][-1]["year"]
            if max_year == max_allowed_year:
                continue
            value_objects = self._get(param, True, year=max_year)
            while max_year < max_allowed_year:
                max_year += 1
                for value_object in value_objects:
                    adjustment[param].append(dict(value_object, **{"year": max_year}))
        self.adjust(adjustment)


class TaxParams(IndexedParameters):
    schema = "schema.json"
    defaults = "defaults.json"
    field_map = {"compatible_data": fields.Nested(CompatibleDataSchema())}

params = TaxParams()
params.extend()
print("EITC celing max year: ", max(map(lambda x: x["year"], params._EITC_c)), "\n")


EITC celing max year:  2028 



Since all parameters have been extended across the year axis, we can activate `array_first` mode. This means that all parameter values will be available as arrays instead of a list of dictionaries, or [value-objects][1]. 


[1]: https://paramtools.readthedocs.io/en/latest/spec.html#value-object

In [34]:
# all parameters can now be converted to arrays.
params.array_first=True
params.set_state()
print("EITC celing as array: ", params._EITC_c)


EITC celing as array:  [[ 487. 3250. 5372. 6044.]
 [ 496. 3305. 5460. 6143.]
 [ 503. 3359. 5548. 6242.]
 [ 506. 3373. 5572. 6269.]
 [ 510. 3400. 5616. 6318.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]
 [ 519. 3461. 5716. 6431.]]


Note that 519, 3461, 5716, and 6431 have been set as the default value for year 2018 to 2028.

Index Parameters
-------------------------------


The previous example shows how to extend parameter values along a given axis, like "year". However, what's really going on is that they are just repeated until the maximum year is reached. Now, it's time to index them at some specified rate.

To "grow" a parameter forward a year, we need to multiply it by one plus the rate at which it is expected to grow or its recorded growth rate. 